In [62]:
'''
This notebook demonstrates two ways to infer the true mean age and true, underlying spread in age of a stellar population
based on observed ages.
'''

from my_routines.mcmc_class import MCMC
import numpy as np
import pandas as pd

class Line(MCMC):                                                                                      
    '''
    This class is meant for fitting a line to the P-L relation. It will fit the following model to the periods:

    log10P = mr * log10R + mm * log10M + c
    
    where P is a Cepheid period in days
    R is in units of the solar radius
    M is in units of the solar mass
    
    
    mr is the slope of the period-radius relation
    mm is the slope of the period-mass relation
    
    This can be related to a P-L relation by way of:
    
    L = R^2 (T/Tsun)^4
    '''
    
    def __init__(self, debug=False, true=False):                                                                      
        '''
        Inputs
        debug: bool
         If debug, will run fake data generated according to the self.debug_mm, self.debug_mr, and self.debug_c values
        '''
        # will inherit some attributes/methods from the MCMC class
        #kwargs = {'plot_dir':'../plots/age_inference/'}
        super(MCMC, self).__init__()
        self.plot_dir = 'plots/'
        
        # number of cores available on your computer
        self.threads = 6
        
        # set up the problem
        self.debug = debug
        # an answer of debug_mass with Sd of self.debug_unc. self.N is the number of fake stars generated, defined below.
        # these should be default be True
        self.debug_c = 1.0
        self.debug_mm = 0.5
        self.debug_mr = 0.75
        self.debug_mt = 0.01
        self.problem = 'cepheid'
        # set up the data
        self.mass = np.array([])
        self.r = np.array([])
        self.p = np.array([])
        self.teff = np.array([]) # !!! not used
        # by a Gaussian distribution of ages, compared to the low-alpha population.
        self.N = len(self.mass)
        if self.debug:
            self.N = 100 # number of fake stars generated in debug case.
            self.mass = self.debug_mass
            self.r = self.debug_r
            self.p = self.debug_p
            self.teff = self.debug_teff
        # MCMC parameters
        self.nwalkers = 20                                                                               
        self.nsteps = 3000 #2000                                                                              
        self.rejection_fraction = 0.8                                                                   
        self.burninsteps = 1000 #1000  

    # this is a required function for MCMC
    def setup_data(self):
        pass
        return

    @property                                                                                             
    def guess(self):                                                                                      
        self._guess = np.array([1.0, 1.0, 1.0, 1.0]) 
        self.names = np.array(['mm', 'mr', 'mt', 'c']) # names of the variables that appear in the corner plots
        return self._guess                                                                                
                                                                                                          
    @property                                                                                             
    def nvariables(self):                                                                                                                         
        self._nvariables = 4
        return self._nvariables   
    
    @nvariables.setter                                                                                    
    def nvariables(self, value):                                                                          
        self._nvariables = value                                                                          
                                                                                                          
    @property                                                                                             
    def debug_mass(self):                                                                             
        self._true_mass = np.log10(np.linspace(2,5,self.N))
        return self._true_mass 

    @property                                                                                             
    def debug_teff(self):                                                                             
        self._true_teff = np.log10(np.linspace(10**3.6,10.**(3.7),self.N))  
        return self._true_teff
    
    @property                                                                                             
    def debug_r(self):                                                                             
        self._true_r = np.log10(np.linspace(50,70,self.N))  
        return self._true_r
    
    @property                                                                                             
    def debug_p(self):                                                                             
        self._true_p = self.debug_mm*self.debug_mass + self.debug_mr*self.debug_r + self.debug_mt*self.debug_teff + self.debug_c

        return self._true_p
    
    def lnprior(self, *args):   
        return 0.0
    
    def p_model(self, *args):
        return args[0]*self.mass + args[1]*self.r + args[2]*self.teff + args[3]
    
    def logl(self, *args):                                                                                
        ''' 
        $$
        \ln\mathcal{L} for the problem:                                                                   
        \ln\mathcal{L} = -0.5(pobserved - agemodel)^T C^{-1} (ageobserved - agemodel)    
        In this case, there are no off-diagonal terms in the covariance (all observed periods are assumed to be independent from each other),
        so this expression is simplified to be:
        \ln\mathcal{L} = \Sigma_i -0.5(pobserved_i - pmodel_i)^2/\sigma_p_i^2
        $$
        '''                                                                                               
        delta = self.p - self.p_model(*args[0])                            
        sigma = 0.000001
        
        # assumes gaussian uncertainties...
        self.chi2 = np.sum(-delta**2/2.0/sigma**2) - 0.5*((np.sum(np.log(sigma**2))))
                     
        # add in the prior
        logl = self.chi2 + self.lnprior(*args[0])                                                                 
                                                                                                          
        if np.isnan(logl):                                                                                
            logl = -np.inf  
            
        return logl    

In [63]:
line = Line(debug=True)
line.run()
#line.plot()

[    0     1     2 ... 11977 11978 11979]
[[0.50000004 0.74999917 0.01000105 0.99999762]
 [0.50000011 0.74999803 0.01000247 0.99999444]
 [0.5000001  0.74999814 0.01000233 0.99999475]
 ...
 [0.50000018 0.74999696 0.01000373 0.99999168]
 [0.50000013 0.74999786 0.01000263 0.99999414]
 [0.50000012 0.74999799 0.01000246 0.99999451]]
False
False
0.00999284302114008
0.9999825014300312
13.40839860241287
^^^^
set best_fit to :
[0.49999999 0.75000012 0.00999986 1.00000032]
set error to:
[9.93995798e-08 1.68733348e-06 2.07490162e-06 4.63590680e-06]
